# Group 1 Advanced Python Group Project Explanation Notebook - FastAPI
This document explains step by step the actions taken in order to develop the second part of the Group Project: FastAPI. The overall structure of the notebook is a little explanation along with the code

This FastAPI builds upon the Flask part of the project, using the same dataset of Orders which revolves around Delivery times. This part serves as an extention, where we created within the database a users table to which the orders are associated to. This way, you could argue this part of the project tries to mimick a "Database system" for orders and clients. **Since you do need to include a API Key in the header, find some examples of valid key below. If using postman, go to authorization > API Key > Key "API-Token" > Value " *one of the tokens***

- 5b61908abd2d87146db48d308c8af7afd49c5ea0d4e6202741c834c485d8504e
- dcf1b02baf475844f16c831da62a13b3d1d2f7f5040d9135f2ec9f4b6695c820
- 4a4d4f07cafac3cbc2dc5922a7d1069c9a671d622b775df819906486cae60fd8

GitHub Repo: https://github.com/felixhommels/mcsbt-gp-fastapi

### Creating the Database and Schemas

In [ ]:
#From database.py

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "sqlite:///database.db"

engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

- Above we created the database with the link to the database file 
- A SQLAlchemy engine is created to connect to the SQLite database
- The SessionLocal serves as a factory with which database sessions can be created throughout other parts of the code

In [ ]:
#From schemas/order.py

from pydantic import BaseModel
from typing import Optional

class OrderCreate(BaseModel):
    order_id: int
    user_id: int
    distance_km: float
    weather: str
    traffic_level: str
    time_of_day: str
    vehicle_type: str
    preparation_time_min: int
    courier_experience_yrs: float
    delivery_time_min: int

class OrderResponse(OrderCreate):
    id: int

    class Config:
        orm_mode = True

- Next we created the schema for the orders, more specifically the response model and the input pydantic model --> this way both the input and the output get validated
- The OrderCreate schema insures that the request body is in line with what the database stores
- The reponse model simply returns the Order ID as a confimation. The Config is done such that SQLAlchemy can directly work with this pydantic model

In [ ]:
#From schemas/user.py

from pydantic import BaseModel
from typing import List

class OrderBase(BaseModel):
    order_id: int

class UserBase(BaseModel):
    name: str
    email: str

class UserCreate(UserBase):
    pass

class UserResponse(UserBase):
    id: int
    orders: List[OrderBase] = []

    class Config:
        orm_mode = True

class UserCreateResponse(UserBase):
    id: int
    api_token: str

    class Config:
        orm_mode = True



- In this section, we defined the schema for users, focusing on both the input and output Pydantic models to ensure data validation.
- The UserCreate schema validates the request body to align with the expected user data structure in the database.
- The UserResponse model returns the user ID and associated orders as part of the response, confirming successful user creation.
- The Config class is set to enable SQLAlchemy to work seamlessly with these Pydantic models, allowing for direct integration with ORM instances.

### Developing the Models

In [ ]:
from sqlalchemy import Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import relationship
from database import Base

class Order(Base):
    __tablename__ = "orders"

    id = Column(Integer, primary_key=True, index=True)
    order_id = Column(Integer, unique=True, index=True)
    distance_km = Column(Float)
    weather = Column(String)
    traffic_level = Column(String)
    time_of_day = Column(String)
    vehicle_type = Column(String)
    preparation_time_min = Column(Integer)
    courier_experience_yrs = Column(Float)
    delivery_time_min = Column(Integer)

    user_id = Column(Integer, ForeignKey("users.id"))
    user = relationship("User", back_populates="orders")

- In this section, we defined the SQLAlchemy model for orders, establishing the structure of the orders table in the database.
- The model includes various fields such as order_id, distance_km, and delivery_time_min, which correspond to the data we want to store.
- The user_id field establishes a foreign key relationship with the users table, linking each order to a specific user.
- The relationship function is used to create a bidirectional relationship between the Order and User models, allowing easy access to a user's orders.
- This setup ensures that our database schema is well-defined and ready for interaction with the FastAPI application.
- Additionally, inheriting from Base is crucial as it enables the ORM functionality, allowing SQLAlchemy to map the model to the corresponding database table seamlessly.

In [ ]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import relationship
from database import Base
import secrets

class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    email = Column(String, unique=True, index=True)
    api_token = Column(String, unique=True, index=True)
    
    orders = relationship("Order", back_populates="user", cascade="all, delete-orphan")

    def generate_api_token(self):
        self.api_token = secrets.token_hex(32)
        return self.api_token


- In this section, we defined the SQLAlchemy model for users, establishing the structure of the users table in the database.
- The model includes fields such as name, email, and api_token, which are essential for user identification and authentication.
- The orders relationship creates a connection to the Order model, allowing easy access to a user's associated orders and enabling cascading operations for deletions.
- The generate_api_token method utilizes the secrets module to create a secure, unique API token for each user, enhancing security for API access.
- This setup ensures that our user schema is well-defined and ready for interaction with the FastAPI application.
- Additionally, inheriting from Base is crucial as it enables the ORM functionality, allowing SQLAlchemy to map the model to the corresponding database table seamlessly.

### Development of App.py

In [ ]:
from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy.orm import Session
from database import get_db
from models.user import User
from models.order import Order
from schemas.user import UserCreate, UserResponse, UserCreateResponse
from schemas.order import OrderCreate, OrderResponse
from typing import List
import uvicorn
from fastapi import Header

app = FastAPI()

In [ ]:
def get_current_user(api_token: str = Header(None), db: Session = Depends(get_db)):
    if not api_token:
        raise HTTPException(status_code=401, detail="Missing API Token")

    user = db.query(User).filter(User.api_token == api_token).first()
    if not user:
        raise HTTPException(status_code=401, detail="Invalid API Token")

    return user

- In this section, we defined the get_current_user function, which serves as a dependency for user authentication in the FastAPI application.
- Within the database, each user has a API Token, with which they can access all endpoints
- The function accepts an api_token from the request headers and a database session, ensuring that the user is authenticated before accessing protected endpoints.
- If the api_token is missing, the function raises an HTTP 401 error with the message "Missing API Token," indicating that authentication is required.
- The function queries the database for a user matching the provided API token. If no user is found, it raises another HTTP 401 error with the message "Invalid API Token," preventing unauthorized access.
- This setup ensures that only authenticated users can access certain endpoints, enhancing the security of the application.
- By using the Depends(get_db) pattern, the function also ensures that it has access to the database session, allowing for efficient user validation.

In [ ]:
@app.post("/create_user", response_model=UserCreateResponse)
def create_user(user: UserCreate, database: Session = Depends(get_db)):
    db_user = User(name=user.name, email=user.email)
    db_user.generate_api_token()
    database.add(db_user)
    database.commit()
    database.refresh(db_user)
    return db_user

- The create_user endpoint allows user registration by accepting a UserCreate model, which includes the user's name and email.
- A new User instance is created, and the generate_api_token method generates a unique API token for the user.
- The user is added to the database, and changes are committed to persist the data.
- The function returns the newly created user as a UserCreateResponse, confirming successful registration.
- This setup ensures secure and efficient user registration using SQLAlchemy's ORM capabilities.

In [ ]:
@app.get("/user/{user_id}", response_model=UserResponse)
def get_user(user_id: int, database: Session = Depends(get_db), current_user: User = Depends(get_current_user)):
    user = database.query(User).filter(User.id == user_id).first()
    if not user:
        raise HTTPException(status_code=404, detail="User not found")
    return user

- The get_user endpoint retrieves user information based on the provided user_id.
- It uses the current_user dependency to ensure the requester is authenticated.
- The function queries the database for the user; if not found, it raises a 404 HTTPException.
- If the user exists, it returns the user data as a UserResponse, providing secure access to user details.

In [ ]:
@app.post("/create_order", response_model=OrderResponse)
def create_order(order: OrderCreate, database: Session = Depends(get_db), current_user: User = Depends(get_current_user)):
    user = database.query(User).filter(User.id == order.user_id).first()
    if not user:
        raise HTTPException(status_code=404, detail="User not found")

    db_order = Order(
        order_id=order.order_id,
        user_id=order.user_id, 
        distance_km=order.distance_km,
        weather=order.weather,
        traffic_level=order.traffic_level,
        time_of_day=order.time_of_day,
        vehicle_type=order.vehicle_type,
        preparation_time_min=order.preparation_time_min,
        courier_experience_yrs=order.courier_experience_yrs,
        delivery_time_min=order.delivery_time_min
    )

    database.add(db_order)
    database.commit()
    database.refresh(db_order)

    return db_order

- The create_order endpoint allows users to create a new order by accepting an OrderCreate model.
- It verifies the user associated with the order using the current_user dependency; if the user is not found, a 404 HTTPException is raised.
- A new Order instance is created with the provided details and added to the database.
- The changes are committed, and the order is refreshed to include any database-generated fields.
- Finally, the function returns the newly created order as an OrderResponse, ensuring secure order creation.

In [ ]:
@app.get("/user/{user_id}/orders", response_model=List[OrderResponse])
def get_user_orders(user_id: int, database: Session = Depends(get_db), current_user: User = Depends(get_current_user)):
    orders = database.query(Order).filter(Order.user_id == user_id).all()
    if not orders:
        raise HTTPException(status_code=404, detail="No orders found for this user")
    return orders

- The get_user_orders endpoint retrieves all orders for a specified user_id.
- It uses the current_user dependency for authentication.
- If no orders are found, a 404 HTTPException is raised.
- If orders exist, they are returned as a list of OrderResponse, providing secure access to the user's orders.

In [ ]:
@app.get("/order/{order_id}", response_model=OrderResponse)
def get_order(order_id: int, database: Session = Depends(get_db), current_user: User = Depends(get_current_user)):
    order = database.query(Order).filter(Order.order_id == order_id).first()
    if not order:
        raise HTTPException(status_code=404, detail="Order not found")
    return order

- The get_order endpoint retrieves a specific order by order_id.
- It uses the current_user dependency for authentication.
- If the order is not found, a 404 HTTPException is raised.
- If found, the order is returned as an OrderResponse, ensuring secure access to order details.